In [1]:
using Flux
using Flux.Data.MNIST
using MLJFlux
using CategoricalArrays
using MLJBase
using MLJ

┌ Info: Recompiling stale cache file /home/ayush99/.julia/compiled/v1.1/Flux/QdkVy.ji for Flux [587475ba-b771-5e3f-ad9e-33799f191a9c]
└ @ Base loading.jl:1184
┌ Info: Recompiling stale cache file /home/ayush99/.julia/compiled/v1.1/MLJFlux/fHjkl.ji for MLJFlux [094fc8d1-fd35-5302-93ea-dabda2abf845]
└ @ Base loading.jl:1184
┌ Warning: Package MLJFlux does not have CategoricalArrays in its dependencies:
│ - If you have MLJFlux checked out for development and have
│   added CategoricalArrays as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with MLJFlux
└ Loading CategoricalArrays into MLJFlux from project dependency, future warnings for MLJFlux are suppressed.



Loading the MNIST dataset:
(Of type ColorTypes.Gray) 

In [2]:
train_labels, train_imgs = CategoricalArray(MNIST.labels()), MNIST.images();

In [3]:
train_imgs[1]

Every neural network should be derived from MLJFlux.Builder. 
The *fit* method just builds the Flux neural network chain

In [4]:
struct densenet <: MLJFlux.Builder
    d1
    d2
end

function MLJFlux.fit(model::densenet, a, b)
    return Chain(vec, Dense(prod(a), model.d1), Dense(model.d1, model.d2), Dense(model.d2, b))
end

In [6]:
builder = densenet(25, 20)

model = ImageClassifier(builder=builder, optimiser = ADAM(0.0003), loss=Flux.mse, n=1)

ImageClassifier(builder = densenet @ 1…71,
                optimiser = ADAM(0.0003, (0.9, 0.999), IdDict{Any,Any}()),
                loss = Flux.mse,
                n = 1,
                batch_size = 1,
                lambda = 0.0,
                alpha = 0.0,
                optimiser_changes_trigger_retraining = false,) @ 1…00

In [7]:
mach = machine(model, train_imgs[1:10], train_labels[1:10]);

In [8]:
fit!(mach)

┌ Info: Training Machine{ImageClassifier} @ 7…61.
└ @ MLJ /home/ayush99/.julia/packages/MLJ/XYSFt/src/machines.jl:135
Optimising neural net:100%[=========================] Time: 0:00:08


Machine{ImageClassifier} @ 7…61


Now, *model* can be used just like any other MLJ model. We can call *fit*, *predict* or *update* methds on this.

We can also do things like hyperparameter tuning with this by creating a machine.

Creating a new model with greater number of epochs will results in continuation of training of the earlier model:

In [9]:
model.n = 2
fit!(mach)

┌ Info: Updating Machine{ImageClassifier} @ 7…61.
└ @ MLJ /home/ayush99/.julia/packages/MLJ/XYSFt/src/machines.jl:147
Optimising neural net:100%[=========================] Time: 0:00:00


Machine{ImageClassifier} @ 7…61


In [10]:
evaluate!(mach)

┌ Info: Evaluating using cross-validation. 
│ nfolds=6. 
│ shuffle=false 
│ measure=MLJ.cross_entropy 
│ operation=StatsBase.predict 
│ Resampling from all rows. 
└ @ MLJ /home/ayush99/.julia/packages/MLJ/XYSFt/src/resampling.jl:137
Cross-validating: 100%[=========================] Time: 0:00:02


6-element Array{Float32,1}:
 1.9640357
 2.4369678
 2.5672576
 2.2667425
 2.5875814
 2.6688087

In [11]:
model.n = 4

4

In [12]:
fit!(mach)

┌ Info: Training Machine{ImageClassifier} @ 7…61.
└ @ MLJ /home/ayush99/.julia/packages/MLJ/XYSFt/src/machines.jl:135
Optimising neural net:100%[=========================] Time: 0:00:00


Machine{ImageClassifier} @ 7…61
